In [1]:
#Script to calculate Gregory regressions on the TRACMIP output (zonal mean of Aqua4xCO2 - AquaControl). 
#This is necessary to obtain forcing and feedback estimates that can be placed into the energy balance model. 

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import voigtColors as vc
voigtColors = vc.voigtColors

/usr/local/python/anaconda3/envs/pangeo3.7/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/python/anaconda3/envs/pangeo3.7/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
#How am I going to do this? 

#Need the monthly mean output from the beginning to the end of the AquaControl simulations. 
#Specifically need OLR and upward and downward SW radiation at TOA (also "rldt" for models that have it). 
#Also need surface temperature. 
#And for breaking down the different forcings and feedbacks, may need all the other TOA & surface energy variables as well.

In [ ]:
#Test loading a file from Michela's climatology: (based on Charles's script)
ds_test = xr.open_dataset('http://fletcher.ldeo.columbia.edu:81/home/.OTHER/.biasutti/.netcdf/.TRACMIP/.AmonClimAug2nd2016/.PP/.ECHAM-6.3/.AquaControl/.hfls_tf.nc/.hfls/dods', decode_times=False)

In [ ]:
ds_test

In [ ]:
#As I suspected this only has 12 times--time averaging has already been done by Michela. 

In [ ]:
#Will instead need to get the data some other way, most likely downloading the Amon files from ESGF.
#I think I have no other choice... Miami repository has the data all in one NetCDF file, for some models.

In [ ]:
#5-14-19: have now downloaded the data (in ../nc_esgf/Amon). Need to figure out how to load everything from the XArray files
#Used Glob for the previous EBM tuning. 
#How about creating a dataset object that contains the zonal means of all the relevant variables for the regressions/feedbacks? 
#(Maintaining monthly values but this will make regressions much easier). 
#Another one for global means perhaps. 
#This way I only have to open the individual model NetCDF files once. 
#But... there is an issue. The data lengths are not the same for each file/model. 
#So, separate dataset for each model then? Don't want to truncate each one...

In [4]:
#Load climatology
ds_climo = xr.open_dataset('../nc_revised_20181130/master.nc')

In [5]:
model_filenames = {'AM2': 'AM21', 
                   'CAM3': 'CAM3', 
                   'CAM4': 'CAM4',
                   'CNRM-AM6-DIA-v2': 'CNRM-AM5', 
                   'ECHAM-6.1': 'ECHAM61', 
                   'ECHAM-6.3': 'ECHAM63', 
                   'GISS-ModelE2': 'GISS-ModelE2',
                   'IPSL-CM5A': 'LMDZ5A', 
                   'MIROC5': 'MIROC5', 
                   'MPAS': 'MPAS', 
                   'MetUM-GA6-CTL': 'MetUM-CTL', 
                   'MetUM-GA6-ENT': 'MetUM-ENT', 
                   'NorESM2': 'CAM5Nor'}

models = ['AM2', 'CAM3', 'CAM4', 'CNRM-AM6-DIA-v2', 'ECHAM-6.1', 'ECHAM-6.3', 'GISS-ModelE2', 'IPSL-CM5A', 
              'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT', 'NorESM2']

In [ ]:
#Loop through the models, load the monthly means for all the relevant variables, take zonal mean, regrid and save as a separate NetCDF file
#For Aqua4xCO2. 
#Note rsdtcs doesn't exist.
#rluscs doesn't either (on repository) but I only need SW variables for APRP.
varlist = ['ts', 'clt', 'rlds', 'rsds', 'rlus', 'rsus', 'rsdt', 'rsut', 'rlut', 'rldscs', 'rsdscs', 'rsuscs', 'rsutcs', 'rlutcs']
varlist_no_rldscs = ['ts', 'clt', 'rlds', 'rsds', 'rlus', 'rsus', 'rsdt', 'rsut', 'rlut', 'rsdscs', 'rsuscs', 'rsutcs', 'rlutcs'] #NorESM is missing this
#for model in models:
#for model in ['NorESM2']:
for model in ['ECHAM-6.1', 'ECHAM-6.3']:
    dsdict = dict()
    for var in varlist:
        if not(model == 'NorESM2' and var == 'rldscs'):
            dsdict[var] = xr.open_dataset(glob.glob('../nc_esgf/Amon/'+var+'_Amon_'+model_filenames[model]+'_aqua4xCO2TRACMIP_r1i1p1_*')[0])
            print('Loaded '+glob.glob('../nc_esgf/Amon/'+var+'_Amon_'+model_filenames[model]+'_aqua4xCO2TRACMIP_r1i1p1_*')[0])
    #Assemble the different variables into one XArray Dataset
    ds_all = xr.merge(dsdict.values())
    #Take zonal mean
    ds_zm = ds_all.mean(dim='lon')
    #Regrid to the common latitude grid of the climatology file
    #interp_like won't really work because the time dimensions don't match up. 
    #Use regular interp (but could get NaNs at boundaries)
    ds_zm_i = ds_zm.interp(coords = {'time': ds_zm.time.data, 'lat': ds_climo.lat.data})
    #Save NetCDF file
    ds_zm_i.to_netcdf('nc_from_xarray/monthly_zm_forGregory_rg_a4_'+model+'.nc')
    
#Ultimately these were regridded in "regrid_data_forGregory.py" script

In [ ]:
#Gregory regression is of TOA change against base state temperature. 
#So I will need that as well. But I only need the climatology for AquaControl, which I have already.

In [ ]:
print(ds_zm)

In [ ]:
#OK, I have now made all these monthly zonal mean files, one for each model. 
#What next? 
#Comment out above code in case I need to rerun the notebook.
#Need to load the original climatology dataset and calculate other relevant zonal means. 
#Then do time regression (Separately for each month? Need to eliminate seasonal cycle somehow) 
#of the net TOA radiation against the climatology (load from existing dataset), to obtain forcing  
#and feedback terms to plug into the EBM.
#Use climatology for the ocean heat storage 

In [ ]:
#What about the GISS model? It's not in the climatology file. But I might be able to add it using the script adapted from Charles.

In [ ]:
#Load all the relevant 
#(Do an example for now)

In [ ]:
#Oh: the grids are not the same for the climatology vs. the monthly means. 
#And is it OK to run the EBM on different grids for each model? 
#There are 3 choices to deal with this: 

#1. Regrid the monthly means to the latitude grid of the climatology
#2. Regrid the climatology back to the original latitude grid of each model 
#3. Download the monthly mean AquaControl data and recalculate climatology

#1 seems easiest. Will have to redo the whole zonal mean loop...

In [ ]:
#Doing this--it takes about 1-2 minutes per model with the regridding (why so long?) and makes the files 3-4 times bigger.
#NorESM2 seems to have failed at the "merge" stage due to a calendar issue. 
#What do the times look like for each NetCDF file for this model? Is one of them different?
#All are "noleap" calendars... is it trying to merge the NorESM with the MetUM files?
#Yes, dsdict has one variable not replaced in that case.

In [ ]:
#Load some of the others to test 
ds_rg_CAM3 = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_rg_a4_CAM3.nc')
ds_CAM3 = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_a4_CAM3.nc')

In [ ]:
ds_rg_CAM3

In [ ]:
ds_rg_CAM3['ts']

In [ ]:
#Check that the regridding was done correctly. Unfortunately I do seem to have nans at the poles. Duplicate these values?


In [ ]:
ds_rg_Nor = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_rg_a4_NorESM2.nc')
ds_Nor = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_a4_NorESM2.nc')

In [ ]:
plt.plot(ds_rg_Nor.lat, ds_rg_Nor['ts'].isel(time=0), 'k')
plt.plot(ds_Nor.lat, ds_Nor['ts'].isel(time=0), 'g')

In [ ]:
#Looks good. Zoom in on the poles
plt.plot(ds_rg_Nor.lat, ds_rg_Nor['ts'].isel(time=0), 'k', linewidth=3)
plt.plot(ds_Nor.lat, ds_Nor['ts'].isel(time=0), 'g', linewidth=3, linestyle='dashed')
plt.axis([-91, -87, 265, 267.2])
#Still looks good.

In [ ]:
print(ds_rg_Nor['ts'].isel(time=0))
#OK, NorESM has no nans. This is because the original had a data point at the poles.

In [ ]:
#Same plots for CAM3
plt.plot(ds_rg_CAM3.lat, ds_rg_CAM3['ts'].isel(time=0), 'k')
plt.plot(ds_CAM3.lat, ds_CAM3['ts'].isel(time=0), 'g')
plt.figure()
plt.plot(ds_rg_CAM3.lat, ds_rg_CAM3['ts'].isel(time=0), 'k', linewidth=3)
plt.plot(ds_CAM3.lat, ds_CAM3['ts'].isel(time=0), 'g', linewidth=3, linestyle='dashed')
#plt.axis([-91, -85, 273, 275])
plt.axis([82, 91, 276, 277])
#OK, original had southernmost point at -88, now cut off to -87.5 with NaNs elsewhere. 
#Similar story at north pole. 

In [ ]:
#So, how to deal with the missing values? 
#Can't put NaNs into the EBM. 
#Instead 
#In the climatology file the 2 edge points pretty much duplicate each other but not the third point. 
#What is being done? Some type of cubic spline?
#Before deciding what to do, print the regridded temperatures from every model. 

In [ ]:
# for model in models:
#     ds_temp = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_rg_a4_'+model+'.nc')
#     print(model)
#     print(ds_temp.isel(time=0)['ts'])
#     print('')
#     print('')
#     print('')
    
#How many Nans on each end for each model?
#AM2: 1
#CAM3: 2
#CAM4: 0
#CNRM: 1
#ECHAM6.1: 1
#ECHAM6.3: 1
#GISS: 1
#IPSL: 0
#MIROC: 1
#MPAS: 0
#MetUM-CTL: 1
#MetUM-ENT: 1
#NorESM: 0

In [ ]:
#Fill all the NaNs with adjacent edge values:
#This will require the weird XArray location indexing stuff. 
models_1nan = ['AM2', 'CNRM-AM6-DIA-v2', 'ECHAM-6.1', 'ECHAM-6.3', 'GISS-ModelE2', 'MIROC5', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT']
models_2nan = ['CAM3']
#for model in models:
for model in ['ECHAM-6.1', 'ECHAM-6.3']:
    ds_temp = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_rg_a4_'+model+'.nc')
    if model in models_1nan:
        print('Filling in nans with adjacent values for model: ' + model)
        #ds_temp[var].loc[dict(lat=-89.5)] = ds_temp[var].sel(lat=-88.5) #This creates error--this might only be possible in DataArrays, not Datasets.
        #So I have to loop through the variables.
        for var in varlist:
            ds_temp[var].loc[dict(lat=-89.5)] = ds_temp[var].sel(lat=-88.5)
            ds_temp[var].loc[dict(lat=89.5)] = ds_temp[var].sel(lat=88.5)
    elif model in models_2nan:
        print('Filling in nans with adjacent values for model: ' + model)
        for var in varlist:
            ds_temp[var].loc[dict(lat=-89.5)] = ds_temp[var].sel(lat=-87.5)
            ds_temp[var].loc[dict(lat=-88.5)] = ds_temp[var].sel(lat=-87.5)
            ds_temp[var].loc[dict(lat=88.5)] = ds_temp[var].sel(lat=87.5)
            ds_temp[var].loc[dict(lat=89.5)] = ds_temp[var].sel(lat=87.5)
    ds_temp.to_netcdf('nc_from_xarray/monthly_zm_forGregory_rg2_a4_'+model+'.nc')

In [ ]:
#Try printing again--did this work?

# for model in models:
#     ds_temp = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_rg2_a4_'+model+'.nc')
#     print(model)
#     print(ds_temp.isel(time=0)['ts'])
#     print('')
#     print('')
#     print('')

#Yes, it worked!

In [ ]:
#Finally have a set of data I can work with. Plan now: 
#Calculate Gregory regression of TOA radiation to get forcing and feedback term
#Calculate the ocean storage term from the climatology files
#Run perturbation EBM (first with 1.27e-6 diffusivity) to see how well this new version captures the polar amplification in TRACMIP

#Then, start calculating and perturbing individual forcings/feedbacks.

#Wait--still haven't figured out the issue of the seasonal cycle.
#Subtract it first?
#Or average it out somehow?
#Maybe: duplicate the climatological variables to match the length of the monthly dataset, 
#       then calculate the anomaly by subtracting these. Then it's just a matter of regressing the two variables together.

In [ ]:
#Function to calculate Gregory regressions for total TOA radiation
#Will run separately for each model.

def GregoryNetRadiationTOA(ds_monthly, ds_climo, model):
    #Calculate variables to regress in monthly values and climatology
    netrad_monthly = -(ds_monthly['rlut']+ds_monthly['rsut']-ds_monthly['rsdt']) #Positive downward
    ts_monthly = ds_monthly['ts']
    netrad_climo = -(ds_climo['rlut']+ds_climo['rsut']-ds_climo['rsdt']).sel(exp='AquaControl', model=model).mean(dim='lon') 
    ts_climo = (ds_climo['ts']).sel(exp='AquaControl', model=model).mean(dim='lon')
    
    #Duplicate the climatology to match the length of the monthly output
    numYears = int(len(ts_monthly.time)/len(ts_climo.time))
    #How to duplicate the climatology variables? 
    #Hmm... maybe it will need to be just a regular NumPy array.
    netrad_climo_raw = netrad_climo.data
    ts_climo_raw = ts_climo.data
    netrad_climo_dupe = np.tile(netrad_climo_raw, (numYears,1))
    ts_climo_dupe = np.tile(ts_climo_raw, (numYears,1))
    
    #Calculate the anomaly by subtracting these variables; this will take care of seasonal cycle
    netrad_anom = netrad_monthly - netrad_climo_dupe
    #Try a resampling, based on XArray documentation for how to compute a seasonal mean. Hopefully calendar treated correctly.
    t = netrad_anom.resample(time='1Y').mean('time').isel(lat=50) #Looks OK
    ts_anom = ts_monthly - ts_climo_dupe
    
    #Take annual averages of the anomaly to get rid of residual seasonal cycle that is causing problems
    #I think this time I should worry about the month lengths.
    #Resample?
    netrad_anom_ann = netrad_anom.resample(time='1Y').mean('time')
    ts_anom_ann = ts_anom.resample(time='1Y').mean('time')
    
    #Debugging printing, plots, etc.
    
    print(numYears) #e.g. 40 for CAM3
    print(np.shape(ts_climo_raw))
    print(np.shape(ts_climo_dupe))
    plt.figure()
    plt.plot(netrad_anom.isel(lat=50))
    plt.plot(np.arange(len(t))*12+6,t, 'k', linewidth=2)
    plt.figure()
    plt.plot(netrad_monthly.isel(lat=50))
    plt.plot(netrad_climo_dupe[:,50])
    plt.figure()
    plt.plot(ts_anom.isel(lat=50))
    plt.figure()
    plt.plot(ts_monthly.isel(lat=50))
    plt.plot(ts_climo_dupe[:,50])
    
    #Regress the net radiation anomaly against the temperature anomaly and return the slope and intercept
    #Um... polyfit is for 1D vectors but I'm trying to run on every latitude. 
    #Could resort to a loop over the latitudes but is there an easy, vectorized way to do linear regression?
    #No built-in function, solutions online involve pandas or sklearn.
    #So might as well do a loop
    #Do this for the annual average version.
    forcing = np.zeros(len(ds_monthly.lat))
    feedback = np.zeros(len(ds_monthly.lat))
    for i in np.arange(len(ds_monthly.lat)):
        p = np.polyfit(ts_anom_ann.isel(lat=i).data, netrad_anom_ann.isel(lat=i).data, 1)
        forcing[i] = p[1] #intercept
        feedback[i] = p[0] #slope
    
    return forcing, feedback

In [ ]:
#Test for one model

forcing_test, feedback_test = GregoryNetRadiationTOA(xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_rg2_a4_CAM3.nc'), ds_climo, 'CAM3')

In [ ]:
print(forcing_test)

In [ ]:
print(feedback_test)

In [ ]:
plt.plot(forcing_test)
#Something's wrong, how can the forcing for CO2 quadrupling be negative most places? 

In [ ]:
plt.plot(feedback_test)

In [ ]:
#From test plots I put into the function: 
#Seasonal cycle has changed between the two runs, so it still shows up and messes up the regression: so much noise. 
#What can I do? 
#Rolling averages? But this will mess up especially at the beginning..
#Just annual averages?
#Yes, it seems annual averages are the standard (e.g. Andrews et al., GRL, 2012). 

In [ ]:
#I think I've finally gotten this working. 
#Run for all the models and save as NetCDF files
#(and then do the "G" term--ocean heat storage)

In [ ]:
#For the G term: what do I need to do? 
#Change in Down - Up at surface in the climatologies,
#for Aqua4xCO2 - AquaControl
#G = (rlds+rsds-rlus-rsus-hfls-hfss)_Aqua4xCO2 - (same)_AquaControl

In [ ]:
#Loop through the models, calculate forcing, feedback and ocean heat storage terms, and save them in NetCDF files via XArray datasets
#Wait: could save just one file with all of them because they're all in the same dimensions. That would make things easier when 
#running the EBM--don't need to load files separately for each model. 
#But G term can be done outside loop

G_precursor = (ds_climo.sel(exp='Aqua4xCO2')-ds_climo.sel(exp='AquaControl'))
G_anom = G_precursor['rlds']+G_precursor['rsds']-G_precursor['rlus']-G_precursor['rsus']-G_precursor['hfls']-G_precursor['hfss']
G_anom_zm_am = G_anom.mean(dim='lon').resample(time='1Y').mean('time')#Try resample again?

In [ ]:
G_anom_zm_am2 = G_anom.mean(dim='lon').mean(dim='time') #Regular time mean--see what the differences are

In [ ]:
(G_anom_zm_am2-G_anom_zm_am).sel(model='MIROC5').plot() #It's the same.

In [ ]:
G_anom_zm_am2

In [ ]:
print(np.shape(G_anom_zm_am2))
print(len(G_anom_zm_am2.lat))

In [ ]:
models_noGISS = ['AM2', 'CAM3', 'CAM4', 'CNRM-AM6-DIA-v2', 'ECHAM-6.1', 'ECHAM-6.3', 'IPSL-CM5A', 
              'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT', 'NorESM2']

In [ ]:
#Now do the loop to calculate the Gregory regressions. 
#Need DataArrays (2D arrays with model, lat coordinates) for each model.
#Then can merge into Dataset.
forcing = np.zeros((len(models_noGISS), len(G_anom_zm_am2.lat))) #13, 180
feedback = np.zeros((len(models_noGISS), len(G_anom_zm_am2.lat)))
print(forcing)
i = 0
for model in models_noGISS:
    ds_temp = xr.open_dataset('nc_from_xarray/monthly_zm_forGregory_rg2_a4_'+model+'.nc')
    forcing[i,:], feedback[i,:] = GregoryNetRadiationTOA(ds_temp, ds_climo, model)
    i = i + 1

In [ ]:
print(forcing)
print(feedback)

In [ ]:
#OK, now turn the forcing and feedback into XArray DataArrays 
#and merge them with the "G" term into a Dataset.
forcing_DA = xr.DataArray(forcing, coords=[('model', models_noGISS), ('lat', G_anom_zm_am2.lat.data)], name='forcing')
feedback_DA = xr.DataArray(feedback, coords=[('model', models_noGISS), ('lat', G_anom_zm_am2.lat.data)], name='feedback')

In [ ]:
forcing_DA

In [ ]:
feedback_DA

In [ ]:
G = G_anom_zm_am2.drop('CaltechGray', dim='model')

In [ ]:
Gr = G.rename('storage')

In [ ]:
Gr

In [ ]:
GregoryTotalForcingFeedbackStorage_a4 = xr.merge([forcing_DA, feedback_DA, Gr])

In [ ]:
GregoryTotalForcingFeedbackStorage_a4

In [ ]:
GregoryTotalForcingFeedbackStorage_a4.to_netcdf('nc_from_xarray/GregoryTotalForcingFeedbackStorage_a4.nc')